<a href="https://colab.research.google.com/github/hamiltonvalerio/DS_CNES_DATASUS/blob/main/CNES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [174]:
import pandas as pd
import numpy as np
import glob
import time
import requests
from io import StringIO
import unicodedata
import re
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


# SELECIONANDO NO GOOGLE DRIVE A TABELA DO MÊS 12/2020 DE NOMES DE EQUIPAMENTOS (tbEquipamento202012.csv) PARA FAZER UM JOIN COM O DATAFRAME de EQUIPAMENTOS TRAZENDO A NOMENCLATURA (Gama Camara, PET/CT)

In [175]:
url_eq_auxiliar = 'https://drive.google.com/file/d/19Qikm47nw3cy7FL6QwZj_4XmVHzz4Xp6/view?usp=sharing'
path_eq_auxiliar = 'https://drive.google.com/uc?export=download&id='+url_eq_auxiliar.split('/')[-2]
df_eq = pd.read_csv(path_eq_auxiliar,sep=';')

filtro_eq = ['Gama Camara', 'PET/CT']

result_eq = df_eq[df_eq['DS_EQUIPAMENTO'].str.contains('|'.join(filtro_eq))]

result_eq

,CO_EQUIPAMENTO,CO_TIPO_EQUIPAMENTO,DS_EQUIPAMENTO
7,1,1,Gama Camara
66,18,1,PET/CT


# ARRAY DE ESTADOS PARA PESQUISA

In [176]:
array_estados = np.array(["AC", "AL", "AM", "AP", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RO", "RS", "RR", "SC", "SE", "SP", "TO"])
pd.options.display.max_columns = None

# FUNÇÃO PARA RETIRAR ACENTOS

In [177]:
def removerAcentosECaracteresEspeciais(palavra):

    # Unicode normalize transforma um caracter em seu equivalente em latin.
    nfkd = unicodedata.normalize('NFKD', palavra)
    palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])

    # Usa expressão regular para retornar a palavra apenas com números, letras e espaço
    return re.sub('[^a-zA-Z0-9 \\\]', '', palavraSemAcento)

    #crédito: https://gist.github.com/boniattirodrigo/67429ada53b7337d2e79

# MONTA O ARRAY DE LINKS DE EQUIPAMENTOS
Todos os arquivos do ano de 2020, mês 12, todos os Estados Brasileiros.

In [178]:
path_eq = 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQ'
array_urls_eq = []
campos_eq = ['CNES','CPF_CNPJ','CODEQUIP']

for est in array_estados:
    array_urls_eq.append(path_eq+""+est+"_2020.csv")
    print(array_urls_eq)


['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAC_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAL_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAL_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAM_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAL_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAM_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAP_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQA

# GERA O DATAFRAME DE EQUIPAMENTOS

In [179]:
timeStartEQ = time.time()
dfs_EQ = [pd.read_csv(url,sep=',', error_bad_lines=False, index_col=False, dtype='unicode', usecols = campos_eq) for url in array_urls_eq]
total_df_EQ = pd.concat(dfs_EQ, ignore_index=True)
tempoDeExecucaoEQ = (time.time() - timeStartEQ)
print('Tempo de execução em minutos: ' + str(tempoDeExecucaoEQ/60))

Tempo de execução em minutos: 1.3449062744776408


In [180]:
total_df_EQ['CODEQUIP'] = total_df_EQ['CODEQUIP'].astype(int)

In [182]:
filtro_cnpj_eq = ['74045626000231']
result_cnpj_eq = total_df_EQ[total_df_EQ['CPF_CNPJ'].str.contains('|'.join(filtro_cnpj_eq))]

result_cnpj_eq

,CNES,CPF_CNPJ,CODEQUIP


In [183]:
total_df_EQ.head()

,CNES,CPF_CNPJ,CODEQUIP
0,3006166,00000000000000,81
1,3006166,00000000000000,84
2,3006166,00000000000000,83
3,3006166,00000000000000,80
4,3382745,00000000000000,81


In [184]:
total_df_EQ.shape

(8596372, 3)

In [185]:
total_df_EQ.dtypes

CNES        object
CPF_CNPJ    object
CODEQUIP     int64
dtype: object

# FILTRA O DATAFRAME DE EQUIPAMENTOS COM OS FILTROS: coluna_codequip.values (1,18) - QUE CORRESPONDEM AOS CODIGOS DOS EQUIPAMENTOS: Gama Camara e PET/CT 

In [186]:
coluna_codequip = result_eq.loc[:,'CO_EQUIPAMENTO'].astype(int)
coluna_codequip.values

array([ 1, 18])

In [187]:
result_df_EQ = total_df_EQ.loc[total_df_EQ['CODEQUIP'].isin(coluna_codequip.values)]

In [188]:
result_df_EQ.shape

(7343, 3)

In [189]:
result_df_EQ = result_df_EQ.rename(columns={'CODEQUIP': 'CODEQUIP_TAB_EQ'})

In [190]:
result_df_EQ

,CNES,CPF_CNPJ,CODEQUIP_TAB_EQ
921,2001977,00624104000177,1
1396,6225012,07616992000125,1
1538,6655653,11467937000198,1
2138,9467610,28685425000109,1
3609,2001977,00624104000177,1
...,...,...,...
8591389,2792451,25053117004828,1
8591436,2468972,25053117005123,1
8593585,7653948,06180514000152,1
8595290,5098246,07720890000155,1


In [191]:
result_df_EQ.to_csv('/drive/My Drive/DS_CNES/result_df_EQ.csv')

# MONTA O ARRAY DE LINKS ESTABELECIMENTOS

In [192]:
path_st = 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/ST'
array_urls_st = []
campos_st = ['CNES','CPF_CNPJ','ALVARA']

for est in array_estados:
    array_urls_st.append(path_st+""+est+"_2020.csv")
    print(array_urls_st)

['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAC_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAL_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAL_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAM_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAL_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAM_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAP_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STA

# GERA O DATAFRAME DE ESTABELECIMENTOS

In [193]:
timeStartST = time.time()
dfs_ST = [pd.read_csv(urlst, sep=',', error_bad_lines=False, index_col=False, dtype='unicode', usecols = campos_st) for urlst in array_urls_st]
total_df_ST = pd.concat(dfs_ST, ignore_index=True)
tempoDeExecucaoST = (time.time() - timeStartST)
print('Tempo de execução em minutos: ' + str(tempoDeExecucaoST/60))

Tempo de execução em minutos: 1.8879082043965658


In [194]:
total_df_ST.shape

(3581555, 3)

In [195]:
filtro_cnpj = ['07738336000103']
result_cnpj = total_df_ST[total_df_ST['CPF_CNPJ'].str.contains('|'.join(filtro_cnpj))]

result_cnpj

,CNES,CPF_CNPJ,ALVARA
533289,3946207,07738336000103,NaN
544312,3946207,07738336000103,NaN
555490,3946207,07738336000103,NaN
566684,3946207,07738336000103,NaN
577899,3946207,07738336000103,NaN
589140,3946207,07738336000103,NaN
600416,3946207,07738336000103,NaN
611757,3946207,07738336000103,NaN
623131,3946207,07738336000103,NaN
634550,3946207,07738336000103,NaN


In [196]:
total_df_ST

,CNES,CPF_CNPJ,ALVARA
0,3006166,00000000000000,NaN
1,3382745,00000000000000,NaN
2,3393984,00000000000000,NaN
3,3638685,00000000000000,NaN
4,5403669,00000000000000,NaN
...,...,...,...
3581550,7170149,00000000000000,36/2019
3581551,7170157,00000000000000,34/2019
3581552,7200447,00000000000000,35/2019
3581553,7970269,13038293000148,02/2018


# FAZ O MERGE (RIGHT JOIN NO SQL) USANDO COMO CHAVE O VALOR CNES E REMOVENDO OS DUPLICADOS

Aqui tenho um resultado de todas as empresas da base CNES que trabalham (ou tenham!?) os equipamentos GAMA CAMARA e PET/CT. Nesta lista podemos conferir se, pelo CNES, temos o Alvará.
Com base nesta lista é possível importar a base de Clientes da Cnen e fazer uma comparação utilizando o CNES ou CNPJ cadastrado. 

In [197]:
df_lista_cnes = pd.merge(total_df_ST, result_df_EQ, on="CNES", how="left").drop_duplicates()
df_lista_cnes = (df_lista_cnes.rename(columns={'CPF_CNPJ_x': 'CNPJ'})).drop(columns=['CPF_CNPJ_y'])
df_lista_cnes['CNPJ'] = df_lista_cnes['CNPJ'].str.replace(".","")
df_lista_cnes['CNPJ'] = df_lista_cnes['CNPJ'].str.replace("/","")
df_lista_cnes['CNPJ'] = df_lista_cnes['CNPJ'].str.replace("-","")
df_lista_cnes['CNPJ'] = df_lista_cnes['CNPJ'].str.replace(" ","")


In [198]:
filtro_cnpj_lista = ['07738336000103']
result_cnpj_lista = df_lista_cnes[df_lista_cnes['CNPJ'].str.contains('|'.join(filtro_cnpj_lista))]

result_cnpj_lista

,CNES,CNPJ,ALVARA,CODEQUIP_TAB_EQ
546841,3946207,07738336000103,NaN,NaN


In [199]:
df_lista_cnes

,CNES,CNPJ,ALVARA,CODEQUIP_TAB_EQ
0,3006166,00000000000000,NaN,NaN
1,3382745,00000000000000,NaN,NaN
2,3393984,00000000000000,NaN,NaN
3,3638685,00000000000000,NaN,NaN
4,5403669,00000000000000,NaN,NaN
...,...,...,...,...
3652875,0446475,36188358000137,2020011950,NaN
3652914,2492644,38143442000160,2020016205,NaN
3653184,6956661,14534761000138,20200009910,NaN
3653415,9510680,27615935000139,2020015539,NaN


In [200]:
df_lista_cnes.to_csv('/drive/My Drive/DS_CNES/df_lista_cnes_leftjoin.csv')

# CARREGA DO GOOGLE DRIVE A LISTA DE CLIENTES DISPONIBILIZADA PELA CNEN
Ao mesmo tempo que carrega a lista, que esta em excel, é formatado o campo de CNPJ para Join com o resultado da base CNES acima.

In [201]:
url_lista_clientes_cnen = 'https://drive.google.com/file/d/1OvY1NsR0Bn8eDq2I28TRvwjQPEdPywAh/view?usp=sharing'
path_lista_clientes_cnen = 'https://drive.google.com/uc?export=download&id='+url_lista_clientes_cnen.split('/')[-2]
df_lista_clientes_cnen = pd.read_excel(path_lista_clientes_cnen, index_col=0).drop(columns=['seq'])
df_lista_clientes_cnen['CNPJ'] = df_lista_clientes_cnen['CNPJ'].str.replace(".","")
df_lista_clientes_cnen['CNPJ'] = df_lista_clientes_cnen['CNPJ'].str.replace("/","")
df_lista_clientes_cnen['CNPJ'] = df_lista_clientes_cnen['CNPJ'].str.replace("-","")
df_lista_clientes_cnen['CNPJ'] = df_lista_clientes_cnen['CNPJ'].str.replace(" ","")
df_lista_clientes_cnen['RAZAOSOCIAL'] = df_lista_clientes_cnen['RAZAOSOCIAL'].apply(lambda x:removerAcentosECaracteresEspeciais(x))
df_lista_clientes_cnen['UNIDADE'] = df_lista_clientes_cnen['UNIDADE'].apply(lambda x:removerAcentosECaracteresEspeciais(x))
df_lista_clientes_cnen['RESPONSAVEL'] = df_lista_clientes_cnen['RESPONSAVEL'].apply(lambda x:removerAcentosECaracteresEspeciais(x))
df_lista_clientes_cnen['ENDERECO'] = df_lista_clientes_cnen['ENDERECO'].apply(lambda x:removerAcentosECaracteresEspeciais(x))
df_lista_clientes_cnen['MUNICIPIO'] = df_lista_clientes_cnen['MUNICIPIO'].apply(lambda x:removerAcentosECaracteresEspeciais(x))

df_lista_clientes_cnen


,CODIGORESP,CNPJ,RAZAOSOCIAL,UNIDADE,RESPONSAVEL,MATRICULACNEN,ENDERECO,NUMERO,COMPLEMENTO,BAIRRO,MUNICIPIO,UF,CEP
CODIGOCLIENTE,,,,,,,,,,,,,
5002184,1,21554423000123,INSTITUTO ONCOLOGICO LTDA JUIZ DE FORAMG,MEDICINA NUCLEAR,GUSTAVO SFFEIR C DE MAGALHAES,16404,RUA SANTOS DUMONT,56,NaN,GRANBERY,JUIZ DE FORA,MG,36010510
5002672,1,33205964000125,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141
5009782,1,15166416000151,REAL SOCIEDADE PORTUGUESA DE BENEFICIENCIA 16 ...,MEDICINA NUCLEAR,EDMARIO ANTONIO G COSTA,11672,AV PRINCESA ISABEL,914,NaN,BARRA AVENIDA,SALVADOR,BA,40140901
5016363,1,75033308000132,CENTRO DE MEDICINA NUCLEAR DO PARANA LTDA SC,MEDICINA NUCLEAR,TAIZA PARENTE ZUKOVSKI,15390,RUA PDE ILDEFONSO,158,NaN,BATEL,CURITIBA,PR,80240160
5017408,1,33804212000180,CLINICA SAO CARLOS SA,SERV MED NUCLEAR,CARLOS AFFONSO FUSER,11663,RUA HUMAITA,296,NaN,BOTAFOGO,RIO DE JANEIRO,RJ,22261001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6251323,1,15126437001549,EMPRESA BRASILEIRA DE SERVICOS HOSPITALARES E...,MEDICINA NUCLEAR,Viviane S Parisotto Marino,14304,AVENIDA ALFREDO BALENA,110,NaN,SANTA EFIGENIA,BELO HORIZONTE,MG,30130100
6251374,1,09609235000230,HOSPITAL CENTRAL DO EXERCITO,MEDICINA NUCLEAR,AFONSO EDUARDO M MUZITANO,16239,RUA FRANCISCO MANUEL,126,NaN,BEFICA,RIO DE JANEIRO,RJ,20911270
6251412,1,15126437002006,EBSERH HUPAAUFAL,MEDICINA NUCLEAR,ANDRE GUSTAVO SILVA PINO,17420,AV LOURIVAL MELO MOTA,S/N,NaN,TABULEIRO DOS MARTINS,MACEIO,AL,57072900


In [202]:
df_lista_clientes_cnen.to_csv('/drive/My Drive/DS_CNES/df_lista_clientes_cnen.csv')

# FAZ O MERGE DA LISTA DE CLIENTES COM A LISTA DA BASE CNES
Utiliza como chave o campo CNPJ

In [203]:
df_resultado_final = pd.merge(df_lista_clientes_cnen, df_lista_cnes, left_on="CNPJ", right_on="CNPJ", how="inner")
df_resultado_final['ALVARA'] = df_resultado_final['ALVARA'].str.replace(",","")
df_resultado_final['COMPLEMENTO'] = df_resultado_final['COMPLEMENTO'].str.replace(",","")

In [204]:
df_resultado_final.to_csv('/drive/My Drive/DS_CNES/df_resultado_final_left.csv')


In [206]:
df1 = dataframe_final
print(df1.shape)
df2 = df_lista_clientes_cnen
print(df2.shape)
df3 = df_resultado_final
print(df3.shape)


(433, 16)
(443, 13)
(508, 16)


In [119]:
df1 = df_lista_clientes_cnen
df2 = pd.merge(dataframe_final, df_lista_clientes_cnen, left_on="CNPJ", right_on="CNPJ", how="inner")
df_all = df1.merge(df2.drop_duplicates(), on=['CNPJ'], 
                   how='left', indicator=True)
df_all

,seq,CODIGORESP,CNPJ,RAZAOSOCIAL,UNIDADE,RESPONSAVEL,MATRICULACNEN,ENDERECO,NUMERO,COMPLEMENTO,BAIRRO,MUNICIPIO,UF,CEP,seq_x,CODIGORESP_x,RAZAOSOCIAL_x,UNIDADE_x,RESPONSAVEL_x,MATRICULACNEN_x,ENDERECO_x,NUMERO_x,COMPLEMENTO_x,BAIRRO_x,MUNICIPIO_x,UF_x,CEP_x,CNES,ALVARA,CODEQUIP,seq_y,CODIGORESP_y,RAZAOSOCIAL_y,UNIDADE_y,RESPONSAVEL_y,MATRICULACNEN_y,ENDERECO_y,NUMERO_y,COMPLEMENTO_y,BAIRRO_y,MUNICIPIO_y,UF_y,CEP_y,_merge
0,0,1,21554423000123,INSTITUTO ONCOLOGICO LTDA JUIZ DE FORAMG,MEDICINA NUCLEAR,GUSTAVO SFFEIR C DE MAGALHAES,16404,RUA SANTOS DUMONT,56,NaN,GRANBERY,JUIZ DE FORA,MG,36010510,0.0,1.0,INSTITUTO ONCOLOGICO LTDA JUIZ DE FORAMG,MEDICINA NUCLEAR,GUSTAVO SFFEIR C DE MAGALHAES,16404.0,RUA SANTOS DUMONT,56,NaN,GRANBERY,JUIZ DE FORA,MG,36010510,2153106,398/2005,1.0,0.0,1.0,INSTITUTO ONCOLOGICO LTDA JUIZ DE FORAMG,MEDICINA NUCLEAR,GUSTAVO SFFEIR C DE MAGALHAES,16404.0,RUA SANTOS DUMONT,56,NaN,GRANBERY,JUIZ DE FORA,MG,36010510,both
1,0,1,33205964000125,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141,0.0,1.0,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595.0,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141,3178447,09/97/027417/2019,18.0,0.0,1.0,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595.0,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141,both
2,0,1,33205964000125,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141,0.0,1.0,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595.0,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141,3178447,050.444/2020,18.0,0.0,1.0,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595.0,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141,both
3,0,1,33205964000125,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141,0.0,1.0,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595.0,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141,3178447,09/97/027417/2019,1.0,0.0,1.0,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595.0,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141,both
4,0,1,33205964000125,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141,0.0,1.0,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595.0,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141,3178447,050.444/2020,1.0,0.0,1.0,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595.0,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,0,1,15126437001549,EMPRESA BRASILEIRA DE SERVICOS HOSPITALARES E...,MEDICINA NUCLEAR,Viviane S Parisotto Marino,14304,AVENIDA ALFREDO BALENA,110,NaN,SANTA EFIGENIA,BELO HORIZONTE,MG,30130100,0.0,1.0,EMPRESA BRASILEIRA DE SERVICOS HOSPITALARES E...,MEDICINA NUCLEAR,Viviane S Parisotto Marino,14304.0,AVENIDA ALFREDO BALENA,110,NaN,SANTA EFIGENIA,BELO HORIZONTE,MG,30130100,0027049,0108706 9930,1.0,0.0,1.0,EMPRESA BRASILEIRA DE SERVICOS HOSPITALARES E...,MEDICINA NUCLEAR,Viviane S Parisotto Marino,14304.0,AVENIDA ALFREDO BALENA,110,NaN,SANTA EFIGENIA,BELO HORIZONTE,MG,30130100,both
563,0,1,09609235000230,HOSPITAL CENTRAL DO EXERCITO,MEDICINA NUCLEAR,AFONSO EDUARDO M MUZITANO,16239,RUA FRANCISCO MANUEL,126,NaN,BEFICA,RIO DE JANEIRO,RJ,20911270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,